In [2]:
import os
import pandas as pd
from tqdm import tqdm
from Scripts.XML_Extraction.Extract_XML import extract_from_xml

In [2]:
dir = r'E:\Brandbank\brandbank_2'
ingredients_out_set = set([])
ingredients=[]
list = os.walk(dir)

outList=[]
oldestoutList=[]
latestoutList = []

columns=[
    'name',
    'gtin',
    'isDelete',
    'date',
    'pl1','pl2','pl3',
    'description',
    'allergyAdvice',
    'recycling',
    'recycling_other',
    'brands',
    'marketing',
    'ingredients',
    'features',
    'storage',
    'preserves']


print('starting')
xml_file_paths=[]
i=0
with tqdm(total=464455) as pbar:  # Do tqdm this way
    for path, subdirs, files in os.walk(dir):
        for name in files:
            if name[-3:] == 'xml':
                if name[-3:] == 'xml':
                    print(name)
                    filepath = os.path.join(path, name)
                    xml_file_paths.append(filepath)
                    productList = extract_from_xml(filepath,name)
                    outList.append(productList)
            pbar.update(1)  # Increment the progress bar

out_df = pd.DataFrame(outList, columns = columns)

starting


  0%|          | 0/464455 [25:29<?, ?it/s]


KeyboardInterrupt: 

In [2]:
import os

# Directory path
dir = r'E:\Brandbank\brandbank'

# Collect all XML file paths
print("Collecting XML file paths...")
xml_file_paths = [os.path.join(path, name) for path, subdirs, files in os.walk(dir) for name in files if name.endswith('.xml')]
print(f"Total XML files found: {len(xml_file_paths)}")


Total XML files found: 861994


In [3]:
import pickle

# Save the list of XML file paths
with open('data/xml_file_paths.pkl', 'wb') as f:
    pickle.dump(xml_file_paths, f)


In [3]:
import pickle
# Load the list of XML file paths
with open('data/xml_file_paths.pkl', 'rb') as f:
    xml_file_paths = pickle.load(f)


In [ ]:
import pandas as pd
from tqdm import tqdm
from concurrent.futures import ThreadPoolExecutor, as_completed

# Define columns for DataFrame
columns = [
    'name', 'gtin', 'isDelete', 'date', 'pl1', 'pl2', 'pl3', 
    'description', 'allergyAdvice', 'recycling', 'recycling_other', 
    'brands', 'marketing', 'ingredients', 'features', 'storage', 'preserves'
]

# Function to process each XML file
def process_single_file(filepath):
    try:
        productList = extract_from_xml(filepath, os.path.basename(filepath))
        return productList
    except Exception as e:
        print(f"Error processing {filepath}: {e}")
        return None

# Output file for batch saving
output_file = 'processed_data.csv'
batch_size = 10000

# Parallel processing with ThreadPoolExecutor
print("Starting parallel processing of XML files...")
with ThreadPoolExecutor(max_workers=12) as executor:  # Adjust max_workers for stability
    future_to_file = {executor.submit(process_single_file, file): file for file in xml_file_paths}

    batch = []
    with tqdm(total=len(xml_file_paths), desc="Processing XML Files") as pbar:
        for future in as_completed(future_to_file):
            result = future.result()
            if result is not None:
                batch.append(result)
            pbar.update(1)

            # Write batch to file and clear batch to avoid memory overload
            if len(batch) >= batch_size:
                batch_df = pd.DataFrame(batch, columns=columns)
                batch_df.to_csv(output_file, mode='a', index=False, header=not os.path.exists(output_file))
                batch = []  # Clear batch after writing

    # Save any remaining data in the final batch
    if batch:
        batch_df = pd.DataFrame(batch, columns=columns)
        batch_df.to_csv(output_file, mode='a', index=False, header=not os.path.exists(output_file))

print("Processing and data saving completed.")


Starting parallel processing of XML files...
Error processing E:\Brandbank\brandbank\00000000006606\13432183.xml: [Errno 13] Permission denied: 'E:\\Brandbank\\brandbank\\00000000006606\\13432183.xml'
Error processing E:\Brandbank\brandbank\0000000013871\1531876.xml: [Errno 13] Permission denied: 'E:\\Brandbank\\brandbank\\0000000013871\\1531876.xml'
Error processing E:\Brandbank\brandbank\0000000015790\2263069.xml: [Errno 13] Permission denied: 'E:\\Brandbank\\brandbank\\0000000015790\\2263069.xml'
Error processing E:\Brandbank\brandbank\00000000273305\12764841.xml: [Errno 13] Permission denied: 'E:\\Brandbank\\brandbank\\00000000273305\\12764841.xml'
Error processing E:\Brandbank\brandbank\0000000035309\13262237.xml: [Errno 13] Permission denied: 'E:\\Brandbank\\brandbank\\0000000035309\\13262237.xml'
Error processing E:\Brandbank\brandbank\00000001008135\12672140.xml: [Errno 13] Permission denied: 'E:\\Brandbank\\brandbank\\00000001008135\\12672140.xml'


Processing XML Files:   0%|          | 0/861994 [00:00<?, ?it/s]


Error processing E:\Brandbank\brandbank\0000000117975\13334650.xml: [Errno 13] Permission denied: 'E:\\Brandbank\\brandbank\\0000000117975\\13334650.xml'
Error processing E:\Brandbank\brandbank\0000000119641\12642794.xml: [Errno 13] Permission denied: 'E:\\Brandbank\\brandbank\\0000000119641\\12642794.xml'
Error processing E:\Brandbank\brandbank\0000000128421\13285806.xml: [Errno 13] Permission denied: 'E:\\Brandbank\\brandbank\\0000000128421\\13285806.xml'
Error processing E:\Brandbank\brandbank\0000000128407\13373236.xml: [Errno 13] Permission denied: 'E:\\Brandbank\\brandbank\\0000000128407\\13373236.xml'
Error processing E:\Brandbank\brandbank\0000000129954\12932791.xml: [Errno 13] Permission denied: 'E:\\Brandbank\\brandbank\\0000000129954\\12932791.xml'
Error processing E:\Brandbank\brandbank\0000000132428\12666297.xml: [Errno 13] Permission denied: 'E:\\Brandbank\\brandbank\\0000000132428\\12666297.xml'
Error processing E:\Brandbank\brandbank\0000000133128\13417901.xml: [Errno 1

In [3]:
out_df

,name,gtin,isDelete,date,pl1,pl2,pl3,description,allergyAdvice,recycling,recycling_other,brands,marketing,ingredients,features,storage,preserves
0,905597.xml,0000000000185,0,2010-08-16,Dairy & Bread,White Bread & Rolls,Other,Waitrose Fig & Raisin Loaf 400g,,,,Waitrose,,,,,
1,643362.xml,0000000000246,0,2006-02-24,Deli,Unallocated,Unallocated,WR Gingerbread biscuit chick each,,,,,,,,,
2,905600.xml,0000000000260,0,2010-08-16,Dairy & Bread,White Bread & Rolls,French Bread/ Rolls,Waitrose Petit Parisienne 290g,,,,Waitrose,,,,,
3,980223.xml,0000000000581,0,2011-02-28,Fruit & Vegetables,Salad Products,Peppers,Waitrose Loose Yellow Peppers,,,,Waitrose,,,,,
4,8445550.xml,00000000006491,0,2020-10-14,Fruit & Vegetables,Apples,Other Apples,Asda Pink Lady Apples,,,,Asda,,,,,
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
157840,2375232.xml,9857423623122,0,2014-08-12,Meat & Fish,Fresh Fish,Salmon,Wrights of Howth Sliced Smoked Salmon 400gr,,,,Wrights of Howth,,"Salmon, Salmo Salar Salt, Oak Smoke",,Keep Refrigerated,
157841,5596613.xml,9993510358956,0,2018-03-19,Meat & Fish,Shellfish,Other Shellfish,Morrisons Fresh Live Scottish Mussels 1kg,,,,Morrisons,,,,"Not Suitable for Home Freezing, Keep Refrigerated",
157842,4607178.xml,9999999999994,0,2017-07-20,Meat & Fish,Lamb,Other Lamb,Glenmór Lamb Carvery Shoulders,,,,Glenmór,,,Irish country meats,,
157843,6771345.xml,9999999999994,0,2019-03-06,Meat & Fish,Reformed Meats,Burgers,Hogan's Farm 12 Buffalo Turkey Burgers and 12 ...,"Sulphur Dioxide/Sulphites, Contains",,,Hogan's Farm,,"Turkey (75%), Buffalo Glaze (Acidity Regulator...","High in protein, Low in fat, Fresh class A tur...",,


In [4]:
out_df.to_csv(r'Data/Bulk Extract Full.csv', index = False)